<a href="https://colab.research.google.com/github/irfankhan745/Git-to-VS-Join/blob/main/auto_clicker_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Project Overview

This notebook demonstrates how to create a simple Flask web application that simulates human-like mouse clicks. The application uses an in-memory SQLite database to manage job settings and utilizes `pyngrok` to expose the Flask API to the internet, making it accessible via a public URL.

### Core Components:

1.  **SQLite Database**: An in-memory database (`:memory:`) is used to store 'click job' settings, including `x`, `y` coordinates, `interval` between clicks, total `clicks`, `randomness`, and `mode` (e.g., 'human').
2.  **Click Simulation Logic**: The `simulate_click_job` function generates a sequence of simulated clicks with human-like randomness in position and timing.
3.  **Job Management Functions**: Functions like `create_job`, `start_job`, and `get_job_status` interact with the SQLite database to manage the lifecycle of click jobs.
4.  **Flask API**: A Flask application provides RESTful endpoints to:
    *   Create new click jobs.
    *   Start existing click jobs and retrieve their simulation logs.
    *   Check the status of a specific click job.
5.  **`pyngrok` Integration**: This library is used to create a secure tunnel from your local Flask server to a public URL, allowing external access to your application.

### Setup and Execution Flow:

1.  **Dependencies**: `flask` and `pyngrok` are installed.
2.  **Database Initialization**: An SQLite database and a `settings` table are created to store job configurations.
3.  **Simulation Logic**: Helper functions (`human_sleep`, `human_offset`, `simulate_click_job`) define the random behavior for clicks.
4.  **Job CRUD**: Functions for creating, starting, and getting the status of jobs are defined.
5.  **Flask App**: The `app` is instantiated, and API routes are defined.
6.  **ngrok Tunnel**: An ngrok tunnel is established to expose the Flask app running on port 5000 to the public internet. The public URL is printed.
7.  **Demonstration**: A sample click job is created, its status is checked, the job is started, and the final status and simulation logs are displayed.

### How to Use:

*   Run all cells in the notebook.
*   Once the `public_url` is printed, you can interact with the Flask API using tools like `curl`, Postman, or by writing client-side code.
*   **API Endpoints:**
    *   `POST /job`: Create a new job. Expected JSON body: `{"x": <int>, "y": <int>, "interval": <float>, "clicks": <int>, "randomness": <float, optional>, "mode": <string, optional>}`
    *   `POST /job/<job_id>/start`: Start a specific job. No request body needed.
    *   `GET /job/<job_id>/status`: Get the status of a specific job.

In [ ]:
!pip install flask

In [ ]:
import sqlite3

conn = sqlite3.connect(":memory:", check_same_thread=False)
cur = conn.cursor()

cur.execute("""
CREATE TABLE settings (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    x INTEGER,
    y INTEGER,
    interval REAL,
    clicks INTEGER,
    randomness REAL,
    mode TEXT,
    status TEXT
)
""")

conn.commit()
print("Database ready")


Database ready


In [ ]:
import time
import random

def human_sleep(base, randomness):
    delta = base * randomness
    return max(0.05, random.uniform(base - delta, base + delta))

def human_offset(randomness):
    offset = int(10 * randomness)
    return random.randint(-offset, offset), random.randint(-offset, offset)

def simulate_click_job(job):
    x, y, interval, clicks, randomness, mode = job

    logs = []
    for i in range(clicks):
        dx, dy = human_offset(randomness)
        sleep_time = human_sleep(interval, randomness)

        logs.append({
            "click": i + 1,
            "position": (x + dx, y + dy),
            "delay": round(sleep_time, 2)
        })

        if mode == "human" and random.random() < 0.05:
            logs.append({"pause": "human break"})

        time.sleep(0.05)  # fast simulation

    return logs


In [ ]:
import time
import random

def human_sleep(base, randomness):
    delta = base * randomness
    return max(0.05, random.uniform(base - delta, base + delta))

def human_offset(randomness):
    offset = int(10 * randomness)
    return random.randint(-offset, offset), random.randint(-offset, offset)

def simulate_click_job(job):
    x, y, interval, clicks, randomness, mode = job

    logs = []
    for i in range(clicks):
        dx, dy = human_offset(randomness)
        sleep_time = human_sleep(interval, randomness)

        logs.append({
            "click": i + 1,
            "position": (x + dx, y + dy),
            "delay": round(sleep_time, 2)
        })

        if mode == "human" and random.random() < 0.05:
            logs.append({"pause": "human break"})

        time.sleep(0.05)  # fast simulation

    return logs


In [ ]:
def create_job(x, y, interval, clicks, randomness=0.3, mode="human"):
    cur.execute("""
    INSERT INTO settings (x,y,interval,clicks,randomness,mode,status)
    VALUES (?,?,?,?,?,?,?)
    """, (x,y,interval,clicks,randomness,mode,"created"))
    conn.commit()
    return cur.lastrowid


def start_job(job_id):
    cur.execute("""
    SELECT x,y,interval,clicks,randomness,mode
    FROM settings WHERE id=?
    """, (job_id,))
    job = cur.fetchone()

    if not job:
        return {"error": "job not found"}

    cur.execute("UPDATE settings SET status='running' WHERE id=?", (job_id,))
    conn.commit()

    logs = simulate_click_job(job)

    cur.execute("UPDATE settings SET status='completed' WHERE id=?", (job_id,))
    conn.commit()

    return logs


def get_job_status(job_id):
    cur.execute("SELECT status FROM settings WHERE id=?", (job_id,))
    row = cur.fetchone()
    return row[0] if row else "not found"


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route("/job", methods=["POST"])
def create():
    data = request.json
    job_id = create_job(
        data["x"],
        data["y"],
        data["interval"],
        data["clicks"],
        data.get("randomness", 0.3),
        data.get("mode", "human")
    )
    return jsonify({"job_id": job_id})


@app.route("/job/<int:job_id>/start", methods=["POST"])
def start(job_id):
    logs = start_job(job_id)
    return jsonify(logs)


@app.route("/job/<int:job_id>/status", methods=["GET"])
def status(job_id):
    return jsonify({"status": get_job_status(job_id)})


In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("389gp6NPKkz8JuKpWHHwi6u260G_5mVB3e8dRFiTmjix6hEbs")


In [ ]:
public_url = ngrok.connect(5000)
print(public_url)


NgrokTunnel: "https://trustable-peevedly-sherley.ngrok-free.dev" -> "http://localhost:5000"


In [ ]:
job_id = create_job(
    x=500,
    y=500,
    interval=1,
    clicks=10,
    randomness=0.35,
    mode="human"
)

print("Job ID:", job_id)
print("Status:", get_job_status(job_id))

logs = start_job(job_id)
print("Final Status:", get_job_status(job_id))
print("Logs:", logs)


Job ID: 1
Status: created
Final Status: completed
Logs: [{'click': 1, 'position': (500, 501), 'delay': 0.99}, {'click': 2, 'position': (497, 497), 'delay': 1.33}, {'click': 3, 'position': (501, 503), 'delay': 0.93}, {'click': 4, 'position': (497, 498), 'delay': 0.76}, {'click': 5, 'position': (498, 501), 'delay': 1.01}, {'click': 6, 'position': (500, 503), 'delay': 1.04}, {'click': 7, 'position': (497, 502), 'delay': 1.24}, {'click': 8, 'position': (498, 503), 'delay': 1.25}, {'click': 9, 'position': (500, 500), 'delay': 1.11}, {'click': 10, 'position': (498, 500), 'delay': 1.13}]


In [ ]:
print(public_url)

NgrokTunnel: "https://trustable-peevedly-sherley.ngrok-free.dev" -> "http://localhost:5000"
